# Token API

In [ ]:
#| default_exp token

In [ ]:
#| export

from typing import Tuple, List, Optional
from dataclasses import dataclass
from sugar.helpers import normalize_address, ADDRESS_ZERO
from sugar.abi import lp_sugar
from sugar.config import SugarConfig

In [ ]:
#| hide
from nbdev.showdoc import *


# load env
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
#| export

@dataclass(frozen=True)
class Token:
    """Data class for Token
    based on: https://github.com/velodrome-finance/sugar/blob/v2/contracts/LpSugar.vy#L17
    """

    token_address: str
    symbol: str
    decimals: int
    listed: bool

    def value_from_bigint(self, value: float) -> float:
        return value / 10**self.decimals

    @classmethod
    def from_tuple(cls, t: Tuple) -> "Token":
        (token_address, symbol, decimals, _, listed) = t
        return Token(
            token_address=normalize_address(token_address),
            symbol=symbol,
            decimals=decimals,
            listed=listed,
        )

    @classmethod
    #@cache_in_seconds(SUGAR_TOKENS_CACHE_MINUTES * 60)
    async def get_all_listed_tokens(cls) -> List["Token"]:
        tokens = await cls.get_all_tokens()
        return list(filter(lambda t: t.listed, tokens))

    @classmethod
    #@cache_in_seconds(SUGAR_TOKENS_CACHE_MINUTES * 60)
    async def get_all_tokens(cls) -> List["Token"]:
        config = SugarConfig.get_config()
        sugar = config.web3.eth.contract(address=config.sugar_contract_addr, abi=lp_sugar[config.protocol_name])
        tokens = await sugar.functions.tokens(config.pagination_limit, 0, ADDRESS_ZERO, []).call()
        return list(map(lambda t: Token.from_tuple(t), tokens))
        

    @classmethod
    async def get_by_token_address(cls, token_address: str) -> Optional["Token"]:
        """Get details for specific token

        Args:
            token_address (str): token address

        Returns:
            Token: target token or None
        """
        try:
            normalized_address = normalize_address(token_address)
            tokens = await cls.get_all_listed_tokens()
            return next(t for t in tokens if t.token_address == normalized_address)
        except Exception:
            return None

## Tests

In [ ]:
from fastcore.test import test_eq

tokens = await Token.get_all_tokens()

test_eq(len(tokens) > 0, True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()